# Análise e Exploração dos Dados

In [1]:
# Importações
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import nltk
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import gensim
import pyLDAvis.gensim

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
# downloads
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

### Importando os datasets

In [ ]:
falsas = pd.read_csv('./datasets/falsas.csv')
falsas.head()

In [ ]:
verdadeiras = pd.read_csv('./datasets/verdadeiras.csv')
verdadeiras.head()

In [ ]:
#df = pd.read_csv('./datasets/merged.csv')

In [ ]:
print('\033[1m' + 'Dataset de Notícias Falsas' + '\033[0m')

print('\nExistem {} observações e {} features. \n'.format(falsas.shape[0], falsas.shape[1]))

categorias = falsas.groupby('Categoria').count()

print(f'Quantidade de registros por categoria:\n {categorias}')

print('-------------------------------------------------')

print('\033[1m' + 'Dataset de Notícias Verdadeiras' + '\033[0m')

print('\nExistem {} observações e {} features. \n'.format(verdadeiras.shape[0], verdadeiras.shape[1]))

categorias = verdadeiras.groupby('Categoria').count()

print(f'Quantidade de registros por categoria:\n {categorias}')

In [ ]:
#?WordCloud

In [ ]:
stopwords = nltk.corpus.stopwords.words("portuguese")
outras_stopwords = ['sobre', 'pode', 'ainda', 'após', 'ano', 'diz', 'disse', 'vai', 'anos', 'país','whatsapp', 'outro', 'desde', 'durante']
stopwords.extend(outras_stopwords)

In [ ]:
textos = ''.join(resumo for resumo in falsas.Resumo)
print ('Existem {} palavras na combinação de todos os resumos.'.format(len(textos)))

In [ ]:

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(textos)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
textos = ''.join(titulo for titulo in falsas.Titulo)
print ('Existem {} palavras na combinação de todos os resumos.'.format(len(textos)))

In [ ]:

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(textos)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
textos = ''.join(resumo for resumo in verdadeiras.Resumo)
print ('Existem {} palavras na combinação de todos os resumos.'.format(len(textos)))

In [ ]:

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(textos)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
falsas['Titulo'].str.len().hist()

In [ ]:
verdadeiras['Titulo'].str.len().hist()

In [ ]:
falsas['Resumo'].str.len().hist()

In [ ]:
verdadeiras['Resumo'].str.len().hist()

Comprimento médio das palavras

In [ ]:
falsas['Titulo'].str.split().\
   apply(lambda x : [len(i) for i in x]). \
   map(lambda x: np.mean(x)).hist()

In [ ]:
verdadeiras['Titulo'].str.split().\
   apply(lambda x : [len(i) for i in x]). \
   map(lambda x: np.mean(x)).hist()

Ngramas

In [ ]:
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]

In [ ]:
top_n_bigrams=get_top_ngram(falsas['Titulo'],2)[:10] 
x,y=map(list,zip(*top_n_bigrams)) 
sns.barplot(x=y,y=x)

In [ ]:
top_n_bigrams=get_top_ngram(verdadeiras['Titulo'],2)[:10] 
x,y=map(list,zip(*top_n_bigrams)) 
sns.barplot(x=y,y=x)

Exploração de modelagem de tópicos com pyLDAvis

In [ ]:
pip install gensim

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import gensim
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim

def preprocessa_noticia(df):
    corpus = []
    stem = PorterStemmer()
    lem = WordNetLemmatizer()
    for news in df['Titulo']:
        words=[w for w in word_tokenize(news) if (w not in stop)]
        
        words=[lem.lemmatize(w) for w in words if len(w)>2]
        
        corpus.append(words)
    return corpus

corpus_falsas = preprocessa_noticia(falsas)
verdadeiras = preprocessa_noticia(verdadeiras)

In [ ]:
# modelo bag of words usando o gensim

dic=gensim.corpora.Dictionary(corpus)
bow_corpus = [dic.doc2bow(doc) for doc in corpus]